In [13]:
!pip install "transformers==4.34.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.23.0" "bitsandbytes==0.41.1" "trl==0.7.4" "safetensors>=0.3.1" ipywidgets wandb --upgrade
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
!pip install ninja packaging
!MAX_JOBS=1 pip install flash-attn --no-build-isolation

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [14]:
# add parent directory to path
import sys
import os

project_dir = os.getcwd()
parent_dir = os.path.dirname(project_dir)
sys.path.insert(0, parent_dir)

In [15]:
# configs
from huggingface_hub import notebook_login
import wandb

model_id = "meta-llama/Llama-2-7b-chat-hf"
output_dir = "llama-7-chat-instruction-int4-fc-dpo"
hub_model_id = f"SebastianS/{output_dir}"
wandb.init(project="function_calling", id=output_dir, entity="sebastiansosa", notes="Trained for 3 epochs on 364 samples of DPO formatted data.")
wandb.define_metric("fc_combine", goal="maximize")
wandb.define_metric("exact_match", goal="maximize")
wandb.define_metric("perplexity", goal="minimize")
wandb.define_metric("epoch")
notebook_login()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [16]:
from gaitor_function_calling.data.data_utils import DataAbstractor, build_data_dpo
import pandas as pd
from datasets import Dataset, DatasetDict

data_abstractor = DataAbstractor("prompts_data.json", "dpo_eval")

dpo_data = build_data_dpo(data_abstractor.raw_data)
dpo_df = pd.DataFrame(dpo_data)
dpo_dataset = Dataset.from_pandas(dpo_df)
dataset_dict = DatasetDict({"train": dpo_dataset})

dataset_dict

No train data found
No test data found
Skipped 320 items


DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 364
    })
})

In [17]:
# get data and convert to Datasets format
import pandas as pd
from datasets import Dataset, DatasetDict
from gaitor_function_calling.data.data_utils import DataAbstractor
from gaitor_function_calling.data.prompting_utils import INSTRUCTION

sft_data_abstractor = DataAbstractor("production_data-without_summary-only_fc.json", "sft")

sft_train_df = pd.DataFrame(sft_data_abstractor.train_data)
sft_test_df = pd.DataFrame(sft_data_abstractor.test_data)

sft_train_dataset = Dataset.from_pandas(sft_train_df)
sft_test_dataset = Dataset.from_pandas(sft_test_df)

sft_dataset_dict = DatasetDict({
    'train': sft_train_dataset,
    'test': sft_test_dataset
})

sft_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 833
    })
    test: Dataset({
        features: ['text'],
        num_rows: 17
    })
})

In [18]:
# setup metrics and evaluation
import json
from transformers import EvalPrediction
from gaitor_function_calling.evaluation.evaluation_utils import FunctionCallingMetric, compute_perplexity, get_logits_and_labels
from gaitor_function_calling.data.prompting_utils import INSTRUCTION, json_arguments_from_prompt
import numpy as np

fc_metric = FunctionCallingMetric()
def config_compute_metrics(tokenizer):
    def compute_metrics(pred: EvalPrediction):
        # pred.predictions are a batch of logits
        # pred.label_ids are a batch of tokens
        token_ids = np.argmax(pred.predictions, axis=-1)
        predictions = tokenizer.batch_decode(token_ids, skip_special_tokens=False)
        labels = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=False)
        
        
    
        # Now use your metric class
        fc_result = fc_metric.run(predictions, labels)  # Implement this method in your class
    
        return {
            "fc_combine": fc_result
        }
    return compute_metrics

def custom_evaluation(eval_dataset, model, tokenizer, epoch, save_prompts_path=False):
    print("Starting custom evaluation.")
    model.eval()  # Set the model to evaluation mode
    results = {}

    fc_results = []
    exact_match_results = []
    perplexity_results = []


    if save_prompts_path:
        try:
            with open(save_prompts_path, "r") as f:
                dpo_data = json.load(f)
        except:
            dpo_data = []


    for idx, example in enumerate(eval_dataset):
        print(f"Example {idx}: ", end="")
        post_message = ""
        
        # Custom Function Calling metric
        prompts = None
        try:
            generated_arguments, expected_arguments, prompts = json_arguments_from_prompt(
                example["text"],
                model,
                tokenizer,
                INSTRUCTION,
                {"idx": idx, "epoch": epoch}
            )
            fc_result = fc_metric.run(generated_arguments, expected_arguments)

            fc_results.append(fc_result)

            if save_prompts_path:
                # add prompts to dpo_data.json
                dpo_data.append({
                    "fc_result": fc_result,
                    **prompts
                })
                with open(save_prompts_path, "w") as f:
                    json.dump(dpo_data, f)
        except Exception as e:
            post_message += f"Error function calling: {e}\n"
            fc_results.append(0)

        # exact match metric
        if prompts:
            exact_match_res = fc_metric._sentence_similarity(prompts["expected_str"].split("[/INST]")[1], prompts["generated_str"].split("[/INST]")[1])
            exact_match_results.append(exact_match_res)
        else:
            exact_match_results.append(0)

        # perplexity metric
        try:
            logits, labels = get_logits_and_labels(example["text"], model, tokenizer)
            perplexity = compute_perplexity(logits[..., :-1, :], labels).item()
            perplexity_results.append(perplexity)
        except Exception as e:
            post_message += f"Error perplexity: {e}\n"
            # perplexity_results.append(float('inf'))

        example_metric = {
            "fc_combine": fc_results[-1],
            "exact_match": exact_match_results[-1],
            "perplexity": perplexity_results[-1] if len(perplexity_results) else None
        }
        print(example_metric)
        if post_message:
            print(post_message)
        
    results["fc_combine"] =  sum(fc_results) / len(fc_results)
    results["exact_match"] =  sum(exact_match_results) / len(exact_match_results)
    results["perplexity"] =  sum(perplexity_results) / len(perplexity_results) if len(perplexity_results) else 1

    return results

In [24]:
# base LLM model and tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
# peft config on the model
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [26]:
# training config
from transformers import TrainingArguments
from trl import DPOTrainer
from peft import LoraConfig

use_flash_attention = False

args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="wandb",
    push_to_hub=True,  # Enable pushing to Hub
    hub_model_id=hub_model_id,  # Hugging Face Hub model ID
    hub_strategy="every_save",  # Push to Hub every epoch
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = DPOTrainer(
    model,
    beta=0.1,
    train_dataset=dataset_dict["train"],
    tokenizer=tokenizer,
    args=args,
    peft_config=peft_config,
    max_length=max_seq_length,
)

/home/sosa.s/.local/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:262: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/sosa.s/.local/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:291: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [27]:
# Training and Evaluation Loop
eval_result = custom_evaluation(sft_dataset_dict["test"], model, tokenizer, -1)
print(f"Evaluation results for epoch {-1}: {eval_result}")
wandb.log({"epoch": -1, **eval_result})

num_epochs = 3
for epoch in range(num_epochs):
    print(f"Begin epoch {epoch}")
    # Train for one epoch
    trainer.train()

    # Evaluate the model with the custom evaluation logic
    eval_result = custom_evaluation(sft_dataset_dict["test"], model, tokenizer, epoch)

    wandb.log({"epoch": epoch, **eval_result})

    # Save the model after each epoch (optional)
    trainer.save_model()

    

    # Log or print evaluation results
    print(f"Evaluation results for epoch {epoch}: {eval_result}\n\n\n")

wandb.finish()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Starting custom evaluation.
Example 0: 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'fc_combine': 0.3577747344970703, 'exact_match': 0.8308500051498413, 'perplexity': 6.818745136260986}
Example 1: {'fc_combine': 0.36043891310691833, 'exact_match': 0.7130554914474487, 'perplexity': 6.76043701171875}
Example 2: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 7.09403657913208}
Error function calling: 'NoneType' object has no attribute 'group'

Example 3: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 9.144186019897461}
Error function calling: 'NoneType' object has no attribute 'group'

Example 4: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 9.144186019897461}
Error function calling: No function call found in generated data

Example 5: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 15.047759056091309}
Error function calling: 'NoneType' object has no attribute 'group'

Example 6: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 15.047759056091309}
Error function calling: No function call found in generated data

Example 7: {'fc_combine': 0, 'exact_match': 0.

/apps/pytorch/2.0.1/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Could not estimate the number of tokens of the input, floating-point operations will not be computed


...

{'loss': 0.1581, 'learning_rate': 0.0002, 'rewards/chosen': -0.5993881225585938, 'rewards/rejected': -2.8494508266448975, 'rewards/accuracies': 0.8999999761581421, 'rewards/margins': 2.2500627040863037, 'logps/rejected': -76.30184936523438, 'logps/chosen': -63.40406036376953, 'logits/rejected': 0.20261335372924805, 'logits/chosen': 0.18054063618183136, 'epoch': 0.22}
{'loss': 0.2087, 'learning_rate': 0.0002, 'rewards/chosen': -3.5650017261505127, 'rewards/rejected': -6.6329498291015625, 'rewards/accuracies': 0.9375, 'rewards/margins': 3.067948341369629, 'logps/rejected': -110.75443267822266, 'logps/chosen': -90.38066101074219, 'logits/rejected': 0.12735788524150848, 'logits/chosen': 0.10654421895742416, 'epoch': 0.44}
{'loss': 0.225, 'learning_rate': 0.0002, 'rewards/chosen': -3.77581524848938, 'rewards/rejected': -7.385890007019043, 'rewards/accuracies': 0.9375, 'rewards/margins': 3.610074281692505, 'logps/rejected': -119.07518005371094, 'logps/chosen': -92.22935485839844, 'logits/rej

/apps/pytorch/2.0.1/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


{'loss': 0.0739, 'learning_rate': 0.0002, 'rewards/chosen': -2.4179434776306152, 'rewards/rejected': -6.360779762268066, 'rewards/accuracies': 0.9666666388511658, 'rewards/margins': 3.9428367614746094, 'logps/rejected': -107.91798400878906, 'logps/chosen': -78.85916137695312, 'logits/rejected': 0.17341794073581696, 'logits/chosen': 0.18636558949947357, 'epoch': 0.22}
{'loss': 0.1209, 'learning_rate': 0.0002, 'rewards/chosen': -2.9129977226257324, 'rewards/rejected': -7.590792655944824, 'rewards/accuracies': 0.9624999761581421, 'rewards/margins': 4.677794456481934, 'logps/rejected': -123.08984375, 'logps/chosen': -76.90034484863281, 'logits/rejected': 0.44831037521362305, 'logits/chosen': 0.44147101044654846, 'epoch': 0.44}
{'loss': 0.0582, 'learning_rate': 0.0002, 'rewards/chosen': -3.15729022026062, 'rewards/rejected': -9.427106857299805, 'rewards/accuracies': 1.0, 'rewards/margins': 6.269815444946289, 'logps/rejected': -143.4191131591797, 'logps/chosen': -88.94710540771484, 'logits/r

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▃▆█
exact_match,█▁▃▁
fc_combine,█▁█▁
perplexity,▁▁▄█
train/epoch,▁▃▅▇█▁▃▅▇█▁▃▅▇█
train/global_step,▁▃▇███████████▃▇███████████▃▅███████████
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
train/logits/chosen,▁▂▂▃▄▃▄▄▄▆█▂
train/logits/rejected,▁▂▂▃▄▃▅▄▄▇█▂
train/logps/chosen,████▆▂▁▄▄▄▂▁
train/logps/rejected,███▇▆▃▃▄▄▂▁▂
